In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import random

In [2]:
ratings = pd.read_csv('./Desktop/ml-latest-small/ratings.csv', error_bad_lines = False, low_memory = False, usecols = ['userId', 'movieId', 'rating'])

In [3]:
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [4]:
movieid_dict = dict(zip(ratings.movieId.unique(), np.arange(len(ratings.movieId.unique())) + 611))
movieid_dict

{1: 611,
 3: 612,
 6: 613,
 47: 614,
 50: 615,
 70: 616,
 101: 617,
 110: 618,
 151: 619,
 157: 620,
 163: 621,
 216: 622,
 223: 623,
 231: 624,
 235: 625,
 260: 626,
 296: 627,
 316: 628,
 333: 629,
 349: 630,
 356: 631,
 362: 632,
 367: 633,
 423: 634,
 441: 635,
 457: 636,
 480: 637,
 500: 638,
 527: 639,
 543: 640,
 552: 641,
 553: 642,
 590: 643,
 592: 644,
 593: 645,
 596: 646,
 608: 647,
 648: 648,
 661: 649,
 673: 650,
 733: 651,
 736: 652,
 780: 653,
 804: 654,
 919: 655,
 923: 656,
 940: 657,
 943: 658,
 954: 659,
 1009: 660,
 1023: 661,
 1024: 662,
 1025: 663,
 1029: 664,
 1030: 665,
 1031: 666,
 1032: 667,
 1042: 668,
 1049: 669,
 1060: 670,
 1073: 671,
 1080: 672,
 1089: 673,
 1090: 674,
 1092: 675,
 1097: 676,
 1127: 677,
 1136: 678,
 1196: 679,
 1197: 680,
 1198: 681,
 1206: 682,
 1208: 683,
 1210: 684,
 1213: 685,
 1214: 686,
 1219: 687,
 1220: 688,
 1222: 689,
 1224: 690,
 1226: 691,
 1240: 692,
 1256: 693,
 1258: 694,
 1265: 695,
 1270: 696,
 1275: 697,
 1278: 698,
 1

In [5]:
ratings = ratings.apply(lambda x : pd.Series(data = [x[0], movieid_dict[x[1]], x[2]]), axis = 1, result_type = 'broadcast')

In [6]:
ratings

,userId,movieId,rating
0,1.0,611.0,4.0
1,1.0,612.0,4.0
2,1.0,613.0,4.0
3,1.0,614.0,5.0
4,1.0,615.0,5.0
...,...,...,...
100831,610.0,3731.0,4.0
100832,610.0,2646.0,5.0
100833,610.0,3732.0,5.0
100834,610.0,2003.0,5.0


In [7]:
edgelist= [(u, i) for u,i in zip(ratings.userId, ratings.movieId)]

In [8]:
G = nx.Graph()

In [9]:
G.add_edges_from(edgelist)

In [10]:
def move():
    return random.random() > 0.1

In [11]:
start = np.random.randint(low = 1, high = G.number_of_nodes())

print('Starting a random walk of lenght ', 10, ' at node ', start)

if nx.is_connected(G):
    print('Graph is connected')
else :
    print('Graph is not connected')

cur = start

for i in range(10):    
    print('Current node is', cur)
    
    if move() == False:
        continue

    neighbors = list(G.neighbors(cur))

    if len(neighbors) == 1:
        cur = neighbors[0]
    else:
        cur = neighbors[int(np.random.randint(low = 0, high = len(neighbors)-1))]

Starting a random walk of lenght  10  at node  2424
Graph is connected
Current node is 2424
Current node is 542.0
Current node is 2204.0
Current node is 477.0
Current node is 9111.0
Current node is 477.0
Current node is 477.0
Current node is 1879.0
Current node is 249.0
Current node is 249.0


In [12]:
Adj  = nx.adjacency_matrix(G).toarray()
print(Adj)

[[0 1 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [13]:
M  = Adj.astype(np.float)
np.sum(M, axis = 0)

array([232., 215.,  52., ...,   1.,   1.,   1.])

In [14]:
for i in range(0, M.shape[0]):
    M[i] /= np.sum(M[i])

In [15]:
M

array([[0.        , 0.00431034, 0.00431034, ..., 0.        , 0.        ,
        0.        ],
       [0.00465116, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01923077, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [18]:
P0, P_prev, P = np.zeros(G.number_of_nodes()), np.zeros(G.number_of_nodes()), np.zeros(G.number_of_nodes())

print('Initial probability distiribution', P0)

P0[int(np.random.randint(low = 0, high = G.number_of_nodes()-1))] = 1

P_prev = P0

for _ in range(100):
    print(_)
    P = 0.1 * P0 + 0.9 * np.dot(M, P_prev)
    print('Probability distribution after' , _ + 1, 'steps', P.round(2), 'Norm', np.linalg.norm(P-P_prev))
    P_prev = P

Initial probability distiribution [0. 0. 0. ... 0. 0. 0.]
0
Probability distribution after 1 steps [0. 0. 0. ... 0. 0. 0.] Norm 0.900002826610972
1
Probability distribution after 2 steps [0. 0. 0. ... 0. 0. 0.] Norm 0.0039230943697543055
2
Probability distribution after 3 steps [0. 0. 0. ... 0. 0. 0.] Norm 0.0030290059171114605
3
Probability distribution after 4 steps [0. 0. 0. ... 0. 0. 0.] Norm 0.000612054877009841
4
Probability distribution after 5 steps [0. 0. 0. ... 0. 0. 0.] Norm 0.0005435209816722988
5
Probability distribution after 6 steps [0. 0. 0. ... 0. 0. 0.] Norm 0.0005152068185154785
6
Probability distribution after 7 steps [0. 0. 0. ... 0. 0. 0.] Norm 0.00046298503988039566
7
Probability distribution after 8 steps [0. 0. 0. ... 0. 0. 0.] Norm 0.00042723502780327914
8
Probability distribution after 9 steps [0. 0. 0. ... 0. 0. 0.] Norm 0.00038430921447704793
9
Probability distribution after 10 steps [0. 0. 0. ... 0. 0. 0.] Norm 0.0003491468867983163
10
Probability distribu

Probability distribution after 88 steps [0. 0. 0. ... 0. 0. 0.] Norm 9.481115771634396e-08
88
Probability distribution after 89 steps [0. 0. 0. ... 0. 0. 0.] Norm 8.533004194464839e-08
89
Probability distribution after 90 steps [0. 0. 0. ... 0. 0. 0.] Norm 7.679703775020182e-08
90
Probability distribution after 91 steps [0. 0. 0. ... 0. 0. 0.] Norm 6.911733397524637e-08
91
Probability distribution after 92 steps [0. 0. 0. ... 0. 0. 0.] Norm 6.220560057770102e-08
92
Probability distribution after 93 steps [0. 0. 0. ... 0. 0. 0.] Norm 5.598504051986122e-08
93
Probability distribution after 94 steps [0. 0. 0. ... 0. 0. 0.] Norm 5.038653646787508e-08
94
Probability distribution after 95 steps [0. 0. 0. ... 0. 0. 0.] Norm 4.5347882821100066e-08
95
Probability distribution after 96 steps [0. 0. 0. ... 0. 0. 0.] Norm 4.0813094539054785e-08
96
Probability distribution after 97 steps [0. 0. 0. ... 0. 0. 0.] Norm 3.673178508521336e-08
97
Probability distribution after 98 steps [0. 0. 0. ... 0. 0